# Introduction
Jack Wilson
11/20/2025

This notebook outlines the analysis, modeling, and prediction of Formula 1 races

# Import Modules

In [1]:
import pandas as pd

# DataFrame Display Options

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

# Pre-Qualifying

## Prep Data

### Import

In [26]:
pre_qual_ex = pd.read_csv('../data/final/f1_data_pre_qual_clean.csv')
pre_qual_ex.shape

(3335, 139)

### Remove Identifiers

In [27]:
pre_qual_ex.head(1)

,race_id,driver_id,circuit_id,team_id,year,round,driver_name,team_name,cumulative_races,cumulative_wins,cumulative_podiums,cumulative_points,avg_finish_last_3,avg_finish_last_5,avg_finish_last_10,position,points,laps_completed,CLAS_rate,NC_rate,DNF_rate,rookie_flag,circuit_name,type,direction,length,turns,elevation,yellow_flag_prob,double_yellow_prob,red_flag_prob,safety_car_prob,vsc_prob,avg_yellow_count,avg_double_yellow_count,avg_red_count,avg_safety_car_deployments,avg_vsc_deployments,avg_sc_laps,avg_vsc_laps,best_time_FP1,best_time_FP2,best_time_FP3,lap_count_FP1,lap_count_FP2,lap_count_FP3,position_FP1,position_FP2,position_FP3,recorded_lap_time_FP1,recorded_lap_time_FP2,recorded_lap_time_FP3,participated_FP1,participated_FP2,participated_FP3,avg_pit_time_last_5,avg_pit_time_last_10,AirTemp_mean,AirTemp_min,AirTemp_max,AirTemp_std,TrackTemp_mean,TrackTemp_min,TrackTemp_max,TrackTemp_std,WindSpeed_mean,WindSpeed_min,WindSpeed_max,WindSpeed_std,Humidity_mean,Humidity_min,Humidity_max,Humidity_std,Pressure_mean,Pressure_min,Pressure_max,Pressure_std,Rainfall_any,Rainfall_mean,used_soft_FP1,avg_pace_soft_FP1,std_pace_soft_FP1,deg_rate_soft_FP1,used_soft_FP2,avg_pace_soft_FP2,std_pace_soft_FP2,deg_rate_soft_FP2,used_soft_FP3,avg_pace_soft_FP3,std_pace_soft_FP3,deg_rate_soft_FP3,used_medium_FP1,avg_pace_medium_FP1,std_pace_medium_FP1,deg_rate_medium_FP1,used_medium_FP2,avg_pace_medium_FP2,std_pace_medium_FP2,deg_rate_medium_FP2,used_medium_FP3,avg_pace_medium_FP3,std_pace_medium_FP3,deg_rate_medium_FP3,used_hard_FP1,avg_pace_hard_FP1,std_pace_hard_FP1,deg_rate_hard_FP1,used_hard_FP2,avg_pace_hard_FP2,std_pace_hard_FP2,deg_rate_hard_FP2,used_hard_FP3,avg_pace_hard_FP3,std_pace_hard_FP3,deg_rate_hard_FP3,used_intermediate_FP1,avg_pace_intermediate_FP1,std_pace_intermediate_FP1,deg_rate_intermediate_FP1,used_intermediate_FP2,avg_pace_intermediate_FP2,std_pace_intermediate_FP2,deg_rate_intermediate_FP2,used_intermediate_FP3,avg_pace_intermediate_FP3,std_pace_intermediate_FP3,deg_rate_intermediate_FP3,used_wet_FP1,avg_pace_wet_FP1,std_pace_wet_FP1,deg_rate_wet_FP1,used_wet_FP2,avg_pace_wet_FP2,std_pace_wet_FP2,deg_rate_wet_FP2,used_wet_FP3,avg_pace_wet_FP3,std_pace_wet_FP3,deg_rate_wet_FP3
0,1,60,1,1,2018,1,Sebastian Vettel,Ferrari,198,47,99,2425.0,2.666667,5.8,5.7,1,25.0,58,0.855,0.14,0.0,False,Australia,Street circuit,Clockwise,5.278,14,8,0.904192,0.814371,0.11976,0.562874,0.413174,4.095808,3.538922,0.161677,0.784431,0.54491,3.065868,1.221557,84.995,84.451,86.067,22,38,15,5,5,1,True,True,True,True,True,True,24.1256,23.7166,19.772059,18.6,21.0,0.540549,24.758824,22.2,28.3,1.247713,0.754412,0.0,1.3,0.27287,86.838235,81.2,92.3,3.268257,1010.220588,1009.6,1011.5,0.523923,True,0.220588,True,91.193429,4.987276,-0.595135,True,94.16775,11.936055,-1.582934,True,86.067,0.0,0.0,True,85.911,0.0,0.0,True,89.304286,1.853079,-0.382151,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,True,95.912,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0


In [28]:
# Drop human-readable identifiers and data leakage columns
columns_to_drop = [
    'year',
    'round',
    'driver_name',
    'team_name',
    'points',
    'laps_completed',
    'circuit_name'
]
pre_qual_target = pre_qual_ex.drop(columns=columns_to_drop)

print(pre_qual_target.shape)
pre_qual_target.head(1)

(3335, 132)


,race_id,driver_id,circuit_id,team_id,cumulative_races,cumulative_wins,cumulative_podiums,cumulative_points,avg_finish_last_3,avg_finish_last_5,avg_finish_last_10,position,CLAS_rate,NC_rate,DNF_rate,rookie_flag,type,direction,length,turns,elevation,yellow_flag_prob,double_yellow_prob,red_flag_prob,safety_car_prob,vsc_prob,avg_yellow_count,avg_double_yellow_count,avg_red_count,avg_safety_car_deployments,avg_vsc_deployments,avg_sc_laps,avg_vsc_laps,best_time_FP1,best_time_FP2,best_time_FP3,lap_count_FP1,lap_count_FP2,lap_count_FP3,position_FP1,position_FP2,position_FP3,recorded_lap_time_FP1,recorded_lap_time_FP2,recorded_lap_time_FP3,participated_FP1,participated_FP2,participated_FP3,avg_pit_time_last_5,avg_pit_time_last_10,AirTemp_mean,AirTemp_min,AirTemp_max,AirTemp_std,TrackTemp_mean,TrackTemp_min,TrackTemp_max,TrackTemp_std,WindSpeed_mean,WindSpeed_min,WindSpeed_max,WindSpeed_std,Humidity_mean,Humidity_min,Humidity_max,Humidity_std,Pressure_mean,Pressure_min,Pressure_max,Pressure_std,Rainfall_any,Rainfall_mean,used_soft_FP1,avg_pace_soft_FP1,std_pace_soft_FP1,deg_rate_soft_FP1,used_soft_FP2,avg_pace_soft_FP2,std_pace_soft_FP2,deg_rate_soft_FP2,used_soft_FP3,avg_pace_soft_FP3,std_pace_soft_FP3,deg_rate_soft_FP3,used_medium_FP1,avg_pace_medium_FP1,std_pace_medium_FP1,deg_rate_medium_FP1,used_medium_FP2,avg_pace_medium_FP2,std_pace_medium_FP2,deg_rate_medium_FP2,used_medium_FP3,avg_pace_medium_FP3,std_pace_medium_FP3,deg_rate_medium_FP3,used_hard_FP1,avg_pace_hard_FP1,std_pace_hard_FP1,deg_rate_hard_FP1,used_hard_FP2,avg_pace_hard_FP2,std_pace_hard_FP2,deg_rate_hard_FP2,used_hard_FP3,avg_pace_hard_FP3,std_pace_hard_FP3,deg_rate_hard_FP3,used_intermediate_FP1,avg_pace_intermediate_FP1,std_pace_intermediate_FP1,deg_rate_intermediate_FP1,used_intermediate_FP2,avg_pace_intermediate_FP2,std_pace_intermediate_FP2,deg_rate_intermediate_FP2,used_intermediate_FP3,avg_pace_intermediate_FP3,std_pace_intermediate_FP3,deg_rate_intermediate_FP3,used_wet_FP1,avg_pace_wet_FP1,std_pace_wet_FP1,deg_rate_wet_FP1,used_wet_FP2,avg_pace_wet_FP2,std_pace_wet_FP2,deg_rate_wet_FP2,used_wet_FP3,avg_pace_wet_FP3,std_pace_wet_FP3,deg_rate_wet_FP3
0,1,60,1,1,198,47,99,2425.0,2.666667,5.8,5.7,1,0.855,0.14,0.0,False,Street circuit,Clockwise,5.278,14,8,0.904192,0.814371,0.11976,0.562874,0.413174,4.095808,3.538922,0.161677,0.784431,0.54491,3.065868,1.221557,84.995,84.451,86.067,22,38,15,5,5,1,True,True,True,True,True,True,24.1256,23.7166,19.772059,18.6,21.0,0.540549,24.758824,22.2,28.3,1.247713,0.754412,0.0,1.3,0.27287,86.838235,81.2,92.3,3.268257,1010.220588,1009.6,1011.5,0.523923,True,0.220588,True,91.193429,4.987276,-0.595135,True,94.16775,11.936055,-1.582934,True,86.067,0.0,0.0,True,85.911,0.0,0.0,True,89.304286,1.853079,-0.382151,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,True,95.912,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0


### One-Hot Encode

In [29]:
# One-hot encode 'type' and 'direction' columns
pre_qual_target = pd.get_dummies(pre_qual_target, columns=['type', 'direction'], prefix=['type', 'direction'])

### Convert Booleans

In [30]:
# Convert all boolean columns to integers
bool_columns = pre_qual_target.select_dtypes(include='bool').columns
for col in bool_columns:
    pre_qual_target[col] = pre_qual_target[col].astype(int)

In [31]:
pre_qual_target.head()

,race_id,driver_id,circuit_id,team_id,cumulative_races,cumulative_wins,cumulative_podiums,cumulative_points,avg_finish_last_3,avg_finish_last_5,avg_finish_last_10,position,CLAS_rate,NC_rate,DNF_rate,rookie_flag,length,turns,elevation,yellow_flag_prob,double_yellow_prob,red_flag_prob,safety_car_prob,vsc_prob,avg_yellow_count,avg_double_yellow_count,avg_red_count,avg_safety_car_deployments,avg_vsc_deployments,avg_sc_laps,avg_vsc_laps,best_time_FP1,best_time_FP2,best_time_FP3,lap_count_FP1,lap_count_FP2,lap_count_FP3,position_FP1,position_FP2,position_FP3,recorded_lap_time_FP1,recorded_lap_time_FP2,recorded_lap_time_FP3,participated_FP1,participated_FP2,participated_FP3,avg_pit_time_last_5,avg_pit_time_last_10,AirTemp_mean,AirTemp_min,AirTemp_max,AirTemp_std,TrackTemp_mean,TrackTemp_min,TrackTemp_max,TrackTemp_std,WindSpeed_mean,WindSpeed_min,WindSpeed_max,WindSpeed_std,Humidity_mean,Humidity_min,Humidity_max,Humidity_std,Pressure_mean,Pressure_min,Pressure_max,Pressure_std,Rainfall_any,Rainfall_mean,used_soft_FP1,avg_pace_soft_FP1,std_pace_soft_FP1,deg_rate_soft_FP1,used_soft_FP2,avg_pace_soft_FP2,std_pace_soft_FP2,deg_rate_soft_FP2,used_soft_FP3,avg_pace_soft_FP3,std_pace_soft_FP3,deg_rate_soft_FP3,used_medium_FP1,avg_pace_medium_FP1,std_pace_medium_FP1,deg_rate_medium_FP1,used_medium_FP2,avg_pace_medium_FP2,std_pace_medium_FP2,deg_rate_medium_FP2,used_medium_FP3,avg_pace_medium_FP3,std_pace_medium_FP3,deg_rate_medium_FP3,used_hard_FP1,avg_pace_hard_FP1,std_pace_hard_FP1,deg_rate_hard_FP1,used_hard_FP2,avg_pace_hard_FP2,std_pace_hard_FP2,deg_rate_hard_FP2,used_hard_FP3,avg_pace_hard_FP3,std_pace_hard_FP3,deg_rate_hard_FP3,used_intermediate_FP1,avg_pace_intermediate_FP1,std_pace_intermediate_FP1,deg_rate_intermediate_FP1,used_intermediate_FP2,avg_pace_intermediate_FP2,std_pace_intermediate_FP2,deg_rate_intermediate_FP2,used_intermediate_FP3,avg_pace_intermediate_FP3,std_pace_intermediate_FP3,deg_rate_intermediate_FP3,used_wet_FP1,avg_pace_wet_FP1,std_pace_wet_FP1,deg_rate_wet_FP1,used_wet_FP2,avg_pace_wet_FP2,std_pace_wet_FP2,deg_rate_wet_FP2,used_wet_FP3,avg_pace_wet_FP3,std_pace_wet_FP3,deg_rate_wet_FP3,type_Race circuit,type_Street circuit,direction_Anti-clockwise,direction_Clockwise,direction_Figure eight
0,1,60,1,1,198,47,99,2425.0,2.666667,5.8,5.7,1,0.855000,0.140000,0.0,0,5.278,14,8,0.904192,0.814371,0.11976,0.562874,0.413174,4.095808,3.538922,0.161677,0.784431,0.54491,3.065868,1.221557,84.995,84.451,86.067,22,38,15,5,5,1,1,1,1,1,1,1,24.1256,23.7166,19.772059,18.6,21.0,0.540549,24.758824,22.2,28.3,1.247713,0.754412,0.0,1.3,0.27287,86.838235,81.2,92.3,3.268257,1010.220588,1009.6,1011.5,0.523923,1,0.220588,1,91.193429,4.987276,-0.595135,1,94.167750,11.936055,-1.582934,1,86.067,0.0,0.0,1,85.9110,0.000000,0.000000,1,89.304286,1.853079,-0.382151,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1,95.912000,0.000000,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,1,0,1,0
1,1,57,1,22,207,62,117,2610.0,5.000000,3.4,2.6,2,0.885167,0.105263,0.0,0,5.278,14,8,0.904192,0.814371,0.11976,0.562874,0.413174,4.095808,3.538922,0.161677,0.784431,0.54491,3.065868,1.221557,84.026,83.931,94.225,27,35,7,1,1,8,1,1,1,1,1,1,23.7408,24.8133,19.772059,18.6,21.0,0.540549,24.758824,22.2,28.3,1.247713,0.754412,0.0,1.3,0.27287,86.838235,81.2,92.3,3.268257,1010.220588,1009.6,1011.5,0.523923,1,0.220588,1,93.152214,12.186640,-0.554316,1,94.567235,16.052836,-1.383569,0,0.000,0.0,0.0,0,0.0000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1,112.275333,15.360884,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,1,0,1,0
2,1,6,1,1,271,20,91,1565.0,3.333333,3.6,6.9,3,0.787546,0.208791,0.0,0,5.278,14,8,0.904192,0.814371,0.11976,0.562874,0.413174,4.095808,3.538922,0.161677,0.784431,0.54491,3.065868,1.221557,84.875,84.214,88.499,23,39,13,4,4,2,1,1,1,1,1,1,24.1078,24.9743,19.772059,18.6,21.0,0.540549,24.758824,22.2,28.3,1.247713,0.754412,0.0,1.3,0.27287

### Save

In [32]:
pre_qual_target.to_csv('../data/final/f1_data_pre_qual_final.csv', index=False)

## Analysis

# Pre-Race